In [3]:
conda install ipywidgets

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [1]:
import argparse
import sys
import os

import pandas as pd
import numpy as np
from renet2.raw import load_documents_vis
from renet2.raw_handler import *

from IPython.display import display, clear_output, HTML
import ipywidgets as widgets

/autofs/bal31/jhsu/home/git/tmp/RENET2_b1/src/renet2/raw_handler.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# visualize target GDA text require the s_doc.txtx file in the RENET2 input,
# check the renet2/parse_data.py scripts.

In [3]:
import sys
sys.argv = ['']
parser = init_parser()
args = parser.parse_args()
get_index_path(args)

# if len(sys.argv[1:]) == 0:
#     parser.print_help()
#     sys.exit(1)
# #print(args)
args.ori_tokenizer = loading_tokenizer(args)
args.token_voc_l = len(args.ori_tokenizer)
print('tokenizer size %d' % (args.token_voc_l))


loading word index from /autofs/bal31/jhsu/home/git/tmp/RENET2_b1/src/renet2/utils/word_index
loaded word index, voc size 82948
tokenizer size 82949


## reading RENET2 predicted gene-disease associations

    important: please specific 
    - parsed date dir (should have s_doc file 
    - the RENET2 predicted rst dir at tar_gad_p_f


In [7]:
# RENET2 input data dir, target GDA file, etc
args.raw_data_dir = "../../_data/data/ft_data/"
tar_gad_p_f = '../../_data/data/ft_gda/gda_rst.tsv'
args.fix_snt_n, args.fix_token_n = 400, 54
print('fix input sentences# %d, tokens# %d, batch size %d' % (args.fix_snt_n, args.fix_token_n, args.batch_size))

fix input sentences# 400, tokens# 54, batch size 64


In [8]:
args.no_cache_file = True
text_path = os.path.join(args.raw_data_dir, args.file_name_doc) 
sentence_path = os.path.join(args.raw_data_dir, args.file_name_snt)
ner_path = os.path.join(args.raw_data_dir, args.file_name_ann)

In [9]:
tar_vis_gda = pd.read_csv(tar_gad_p_f, sep='\t')
tar_vis_gda['pmid'] = tar_vis_gda['pmid'].astype(str)
tar_vis_gda['geneId'] = tar_vis_gda['geneId'].astype(str)
tar_vis_gda['diseaseId'] = tar_vis_gda['diseaseId'].astype(str)
_s = tar_vis_gda[['pmid', 'geneId', 'diseaseId']].values.tolist()

In [10]:
all_ori_seq, ner_df, session_info, ori_ner, all_sentence, all_session_map = load_documents_vis(text_path, sentence_path, ner_path, args.ori_tokenizer, args)

loading sorce info
read 500 doc, max token len 0 42



In [11]:
# from IPython.display import HTML as html_print



def cstr_p(s, color1='#4fc1e9', color2='#4fc1e9'):
    return '<p style="color:{};background-color:{}">{}</p>'.format(color1, color2, s)

def add_link(s, _type, _id):
    if _type == 'g':
        return '<a href="https://www.ncbi.nlm.nih.gov/gene/{}" target="_blank" style="text-decoration: none">{}</a>'.format(_id, s)
    else:
        return '<a href="https://meshb.nlm.nih.gov/record/ui?ui={}" target="_blank" style="text-decoration: none">{}</a>'.format(_id, s) 

def cstr_g(s, ck=False):
    if ck:
        return '<span style="color:#673ab7;background-color:#e1bee7;border-style:solid;border-width:1px">{}</span>'.format(s)
    else:
        return '<span style="color:#673ab7;background-color:#e1bee7;">{}</span>'.format(s)

def cstr_d(s, ck=False):
    if ck:
        return '<span style="color:#ff9800;background-color:#ffe0b2;border-style:solid;border-width:1px">{}</span>'.format(s)
    else:
        return '<span style="color:#ff9800;background-color:#ffe0b2">{}</span>'.format(s)
    
def cstr_b(s, color1='#4fc1e9', color2='#4fc1e9'):
    return '<span style="color:{};background-color:{}">{}</span>'.format(color1, color2, s)

def print_text_s(tar_pmid, txt_lst, tar_g, tar_d, tar_s_l, tar_map_i):
    rst_s = []
    for _i, snt in enumerate(txt_lst):
        tmp_s = ''
        for tk in snt:
            if type(tk)==type(''):
                tmp_s += tk
            else:
#                 'gene'
#                 print(tk, tk[0][0][1:-1])
                if tk[0][0][1:-1].isnumeric():
#                     print(tk[1])
                    _tar_s = ' '.join(tk[1])
                    tmp_s += add_link(cstr_g(_tar_s, tk[0][0][1:-1] == tar_g), 'g', tk[0][0][1:-1])
                else:
                    _tar_s = ' '.join(tk[1])
                    tmp_s += add_link(cstr_d(_tar_s, tk[0][0][1:-1] == tar_d), 'd', tk[0][0][1:-1])
#                     tmp_s += cstr_d(_tar_s, tk[0][0][1:-1] == tar_d)
#                 tmp_s += '[' +  tk[1][0] + '] ' + tk[0][0]
            tmp_s += ' '
        n_i = tar_map_i[1][_i]
        if n_i in tar_s_l:
            if len(rst_s) == 0:
                rst_s.append([tar_map_i[0][n_i], tmp_s])
            else:
                if tar_map_i[0][n_i] == rst_s[-1][0]:
                    rst_s[-1][1] = rst_s[-1][1] + cstr_b('_', '#4fc1e9') + tmp_s
                else:
                    rst_s.append([tar_map_i[0][n_i], tmp_s])
    html_rst = ''
    html_rst += '<p>{}</p>'.format('PMID: '+tar_pmid)
    for _tar_s in rst_s:
#         print(_tar_s[0], _tar_s[1])
#         print(cstr(' '.join([cstr(_tar_s[0]), _tar_s[1]])))
#         print(' '.join([cstr_p(_tar_s[0], '#457B9D', '#A8DADC'), _tar_s[1]]))
        html_rst += ' '.join([cstr_p(_tar_s[0], '#457B9D', '#A8DADC'), _tar_s[1]])
#         return rst_s
        
#         break
    return html_rst
    
def on_button_clicked_l(b):
    global i, ii, display_vals
    clear_output()
    if i > 0:
        i -= 1
    print('checking [%d/%d] GDA, in this articles, have [%d/%d] GDA' \
          % (ii+1, len(_s), i+1, len(display_vals)))
#     print(display_vals[i])
#     display(button1, button2)
    display(widgets.HBox((button1p, button2p, button1, button2)))
#     display(button1p, button2p, button1, button2)
    display_html_i(i)
    
def on_button_clicked_r(b):
    global i, ii, display_vals
    clear_output()
    if i < len(display_vals)-1:
        i += 1
    print('checking [%d/%d] GDA, in this articles, have [%d/%d] GDA' \
          % (ii+1, len(_s), i+1, len(display_vals)))
#     print(display_vals[i])
#     display(button1, button2)
    display(widgets.HBox((button1p, button2p, button1, button2)))
#     display(button1p, button2p, button1, button2)
    display_html_i(i)
    
def on_button_clicked_lp(b):
    global i, ii, display_vals
    clear_output()
    if ii > 0:
        ii -= 1
        update_p(_s[ii])
    print('checking [%d/%d] GDA, in this articles, have [%d/%d] GDA' \
          % (ii+1, len(_s), i+1, len(display_vals)))
#     print(_s[ii])
#     display(button1p, button2p, button1, button2)
    display(widgets.HBox((button1p, button2p, button1, button2)))
    display_html_i(i)
    
def on_button_clicked_rp(b):
    global i, ii, display_vals
    clear_output()
    if ii < len(_s)-1:
        ii += 1
        update_p(_s[ii])
    print('checking [%d/%d] GDA, in this articles, have [%d/%d] GDA' \
          % (ii+1, len(_s), i+1, len(display_vals)))
#     print(_s[ii])
#     display(button1p, button2p, button1, button2)
    display(widgets.HBox((button1p, button2p, button1, button2)))
    display_html_i(i)
    
from IPython.display import HTML as html_print


def display_html(tar_pmid, tar_g, tar_d, tar_s_l, tar_map_i):
#     print(tar_s_l, tar_map_i)
#     tar_s_l = [5, 6]
    rst_s = print_text_s(tar_pmid, all_ori_seq[tar_pmid], tar_g, tar_d, tar_s_l, tar_map_i)
    display(HTML(rst_s))
#     return html_print(rst_s)

def display_html_i(i):
#     global tar_pmid, tar_g, tar_d, tar_s, tar_p, display_vals, i
    tar_g, tar_d, tar_s_l = tuple(tar_p[i])
    if 1 in tar_s_l:
        tar_s_l = [0] + tar_s_l
    tar_map_i = all_session_map[tar_pmid]
#     print(tar_s_l)
    print(tar_pmid, tar_g, tar_d, tar_s_l)
    display_html(tar_pmid, tar_g, tar_d, tar_s_l, tar_map_i[:])
    
def update_t(tar_pmid):
    tar_s = ner_df[ner_df['pmid']==tar_pmid]
    

    tar_info = {}
    tar_info['Gene'] = {}
    tar_info['Disease'] = {}
    for index, row in tar_s.iterrows():
        if row['id'] in tar_info[row['type']]:
            tar_info[row['type']][row['id']].append(row['sessions'])
        else:
            tar_info[row['type']][row['id']] = [row['sessions']]

    tar_p = []
    for _g in tar_info['Gene']:
        for _d in tar_info['Disease']:
            a = tar_info['Gene'][_g]
            b = tar_info['Disease'][_d]
            tar_s_l = sorted(list(set(a) & set(b)))
            if len(tar_s_l) > 0:
    #             tar_p.extend([[_g, _d, _i] for _i in tar_s_l])
                tar_p.extend([[_g, _d, tar_s_l]])
    tar_p = sorted(tar_p, key = lambda x: (x[0], x[1], x[2]))
    return tar_s, tar_p

def update_p(s):
    global tar_pmid, tar_g, tar_d, tar_s, tar_p, display_vals, i
#     print(s)
    tar_pmid, tar_g, tar_d = tuple(s)
    tar_s, tar_p = update_t(tar_pmid)
    display_vals = tar_p
    
    
    i = 0
    for _i in range(len(tar_p)):
        if (str(tar_p[_i][0]) == tar_g) and (str(tar_p[_i][1]) == tar_d):
            i = _i
            break

In [12]:
ii = 1234
update_p(_s[ii])
        
button1p = widgets.Button(description="prev_p")
button2p = widgets.Button(description="next_p")
button1 = widgets.Button(description="prev")
button2 = widgets.Button(description="next")



# print(_s[ii], display_vals[i])
button1p.on_click(on_button_clicked_lp)
button2p.on_click(on_button_clicked_rp)
button1.on_click(on_button_clicked_l)
button2.on_click(on_button_clicked_r)

print('checking [%d/%d] GDA, in this articles, have [%d/%d] GDA' \
          % (ii+1, len(_s), i+1, len(display_vals)))
display(widgets.HBox((button1p, button2p, button1, button2)))

display_html_i(i)

checking [1241/1551] GDA, in this articles, have [18/93] GDA


32211513 10273 D005911 [2]
